In [ ]:
#### IMPORT LIBS AND FXS ####
import os, sys, glob, subprocess
import re
import itertools
import numpy as np
np.random.seed(31415)
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import ticker
import pysam
from pyfaidx import Fasta
from Bio.Seq import Seq
# Supress warnings
import warnings
warnings.filterwarnings('ignore')

## DEFINE CUSTOM FUNCTIONS

# PREPROCESSING FUNCTIONS
def count_mapped_reads(bam_file):
    cmd = f"samtools view -c -F 4 -@ 12 {bam_file}"
    return int(subprocess.check_output(cmd, shell=True).decode().strip())
    
def sort_bed_file(input_bed_path, output_bed_path, window = 0):
    """
    Sorts a BED file by chromosome and start position, placing numeric chromosomes first,
    followed by X, Y, and M chromosomes.
    "window" parameter (bp) allows modification of flanking window around previous regex search.
    """

    # Load the BED file
    bed_df = pd.read_csv(input_bed_path, sep="\t", header=None, names=["chr", "start", "end", "hit"])

    # Filter for chromosomes containing 'chr'
    bed_df = bed_df.loc[bed_df["chr"].str.contains("chr")]

    # Custom sorting key
    def chromosome_sort_key(chrom):
        chrom = chrom.replace("chr", "")
        if chrom.isdigit():
            return (0, int(chrom))  # Numeric chromosomes
        elif chrom == "X":
            return (1, 23)  # X chromosome
        elif chrom == "Y":
            return (2, 24)  # Y chromosome
        elif chrom == "M":
            return (3, 25)  # Mitochondrial chromosome
        else:
            return (4, chrom)  # Any other chromosomes

    # Sort DataFrame using the custom key
    bed_df['chr_sort_key'] = bed_df['chr'].map(chromosome_sort_key)
    bed_df["start"] = bed_df["start"].apply(lambda x: x-window)
    bed_df["end"] = bed_df["end"].apply(lambda x: x+window)
    bed_df.sort_values(by=["chr_sort_key", "start"], inplace=True)

    # Drop the temp sort key column
    bed_df.drop(columns=['chr_sort_key'], inplace=True)

    # Save the sorted BED file
    bed_df[["chr", "start", "end", "hit"]].to_csv(output_bed_path, index=False, header=False, sep="\t")
    
# SIGNAL-TO-NOISE RATIO
def calculate_signal_to_noise(bam_path, peaks_df):
    bamfile = pysam.AlignmentFile(bam_path, "rb")
    
    # Number of reads in peaks
    reads_in_peaks = 0
    for index, row in peaks_df.iterrows():
        if not row['Chrom'].startswith("chr"): # SKIP Alternate Assemblies
            continue
        reads_in_peaks += bamfile.count(row['Chrom'], row['Start'], row['End'])
    
    # Total number of reads in BAM file
    total_reads = bamfile.mapped
    
    # Number of reads outside peaks
    reads_outside_peaks = total_reads - reads_in_peaks
    
    bamfile.close()
    
    # Calculate Signal-to-Noise ratio
    signal_to_noise = reads_in_peaks / reads_outside_peaks if reads_outside_peaks > 0 else 0
    return signal_to_noise

## RPKM PROCESSING
# Pattern Matching
def find_hits_and_create_bed(editor, regexes, window = 0, bed_path = f"hits.bed"):
    '''
    Locate Seed&PAM matches in reference genome using regex, then output regions in bed file.
    '''
    genome_path = "/groups/doudna/team_resources/shared_databases/human_reference_genomes/hg38/GENCODE_GRCh38p14_v44/GRCh38_PRI.fa"
    fasta = Fasta(genome_path)
    hits = []

    for chrom in fasta.keys():
        sequence = str(fasta[chrom])
        for regex in regexes:
            for match in re.finditer(regex, sequence):
                start = max(match.start() - window, 0)
                end = min(match.end() + window, len(sequence))
                hits.append((chrom, start, end, match.group()))

    with open(bed_path, 'w') as f:
        for chrom, start, end, match in hits:
            f.write(f"{chrom}\t{start}\t{end}\t{match}\n")

def reverse_complement(seq):
    return str(Seq(seq).reverse_complement())
            
def calculate_rpkm(file_path, scale_factor):
    '''
    Calculate rpkm from coverage bed file.
    '''
    if "complement" in file_path:
        df = pd.read_csv(file_path, sep="\t", header=None, names=["chr", "start", "end", "reads"])
    else:
        df = pd.read_csv(file_path, sep="\t", header=None, names=["chr", "start", "end", "hit", "reads"])

    df['rpkm'] = df['reads'] / (df['end'] - df['start']) * scale_factor
    return df
    
def process_chip_data(glob_path, mapped_bam_reads, experimental_setup):
    '''
    Collect RPKM for samples based on "experimental_setup" dictionary
    '''
    treatment_dict = {}

    if type(glob_path) == list:
        to_process = glob_path
    else:
        to_process = glob.glob(glob_path)
    
    for hit_cov in to_process:
        # Clean file name
        base_name = hit_cov.split("/")[-1].split("_sorted")[0]
        casid = base_name.split("_pseudo")[0].replace("ChIP__KMW_060324_", "").replace("_UCLA", "").replace("d", "")
        regexid = hit_cov.split("/")[-1].split("___")[1]
        treatment = f"{casid}_{regexid}"
        
        print(f"# Processing: {base_name}")
        if os.path.exists(hit_cov):
            print(f"# Processing: {hit_cov}\n{base_name}")
            coverage_df = calculate_rpkm(hit_cov, mapped_bam_reads[base_name] / 1e6)
            coverage_df.columns = [f"{col}_{base_name}___{treatment}" for col in coverage_df.columns]
            
            if treatment in treatment_dict:
                treatment_dict[treatment].append(coverage_df)
            else:
                treatment_dict[treatment] = [coverage_df]

    results = []
    for treatment, files_list in experimental_setup.items():
        treatment_dfs = []
        control_dfs = []
        print(f"# REPLICATES: {treatment}, {files_list}")
        
        # Process treatment replicates
        if len(files_list)>0:
            dfs = treatment_dict.get(files_list[0], [])
            concat_df_trt = pd.concat(dfs, axis=1)
            mean_rpkm = concat_df_trt.filter(like='rpkm').mean(axis=1)
            treatment_dfs.append(mean_rpkm)
        
        # Optionally process control/background replicates
        if len(files_list)>1:
            dfs = treatment_dict.get(files_list[1], [])
            concat_df = pd.concat(dfs, axis=1)
            mean_rpkm = concat_df.filter(like='rpkm').mean(axis=1)
            control_dfs.append(mean_rpkm)

        # Calculate statistics
        treatment_avg = concat_df_trt.filter(like='rpkm').mean(axis=1) 
        control_avg = control_dfs[0]

        # Subtract control from treatment if control data exists
        if not control_dfs:
            treatment_std = concat_df_trt.filter(like='rpkm').mean().std() if treatment_dfs else None
            results.append({'Treatment': treatment, 'Avg_RPKM': treatment_avg.mean(), 'Std_RPKM': treatment_std})
        else:
            rpkm_df = concat_df_trt.filter(like='rpkm')
            for col in rpkm_df.columns:
                rpkm_df[col] = rpkm_df[col] - control_avg
            results.append({'Treatment': treatment, 'Avg_RPKM': rpkm_df.mean(axis=1).mean(), 'Std_RPKM': rpkm_df.mean().std()})

    return pd.DataFrame(results)

# Plotting
def get_color(sample_id):
    '''
    Set color for bar plots
    '''
    if "SpRY" in sample_id:
        return '#9400D3'
    elif "WT" in sample_id:
        return '#A9A9A9'  
    else:
        return 'lightgrey'
    
def plot_bar(ax, xs, ys, title, ylabel, get_color, yerr=None, capsize=5):
    x_pos = np.arange(len(xs))
    colors = [get_color(sample_id) for sample_id in xs]
    
    if yerr is not None:
        ax.bar(x_pos, ys, color=colors, yerr=yerr, capsize=capsize, 
               error_kw={'ecolor': 'black', 'linewidth': 1})
    else:
        ax.bar(x_pos, ys, color=colors)
    
    ax.set_title(title)
    ax.set_xticks(x_pos)
    ax.set_xticklabels(xs, rotation=90, fontsize=8)
    ax.ticklabel_format(style='plain', axis='y')
    ax.yaxis.set_minor_locator(ticker.AutoMinorLocator())
    ax.set_ylabel(ylabel)
    
    if yerr is not None:
        ymax = max(np.array(ys) + np.array(yerr))
        ax.set_ylim(0, ymax * 1.1)

In [2]:
#### SETUP ####

# OUTPUT DIR
out_dir = "/groups/doudna/projects/mtrinidad_projects/ChIPSeq_HSKW/KMW_060324_ChIP/Pseudo_Analysis/"

# PEAK DIR
peak_dir = f"{out_dir}Peaks/"
peak_suffix = "_peaks.narrowPeak"

# SAMPLE IDs: Fastq prefixes
sample_ids = ['ChIP__KMW_060324_dSpRY_UCLA', 
              'ChIP__KMW_060324_dSpRYapo_UCLA',
              'ChIP__KMW_060324_dWT_UCLA', 
              'ChIP__KMW_060324_dWTapo_UCLA'
             ]

## COLLECT MAPPED READ COUNTS FROM FULL-FASTQ ALIGNMENTS
# FULL BAMS
bam_suffix = "_sorted.bam"
bam_dir = "/groups/doudna/projects/mtrinidad_projects/ChIPSeq_HSKW/KMW_060324_ChIP/Alignments/"
bams = [bam_dir+x for x in os.listdir(bam_dir) if x.endswith("_sorted.bam") and x.split("/")[-1].replace(bam_suffix, "") in sample_ids]

# Get mapping stats
mapped_dict = {}
for bam in bams:
    base_id = bam.split("/")[-1].replace(bam_suffix, "")
    print(base_id)
    mapped_reads = count_mapped_reads(bam)
    mapped_dict[base_id] = mapped_reads


In [ ]:
#### CREATE PSEUDO-REP ALIGNMENTS AND PEAKS: Create pseudoreps, align, dedup bams, Macs2 peak-calling ####

## SET VARIABLES
n_reps = 3 # Number pseudoreps
threads='$SLURM_CPUS_ON_NODE'
repair_dir = "/shared/software/bbmap/latest/repair.sh" # bbmap repair script to enforce fastq read order
pseudo_bam_dir = f"{out_dir}Pseudo_Alignments/"
dedup_dir = f"{pseudo_bam_dir}/DEDUP/"
dedup_suffix = "_nodups.bam"

# GENERATE SEEDS
#seeds = [np.random.randint(100, 100 * (len(sample_ids)*n_reps+1)) for _ in range(0,(len(sample_ids)*n_reps+1))]
seeds = [1496, 1729, 1875, 641, 627, 136, 101, 1370, 172, 1246, 1788, 1402, 942, 1488, 345, 1230, 1535, 611, 1292]
print(f"# SEEDS: {seeds}")

# Macs2 dir
macs2_dir = "/home/mtrinidad/miniconda3/envs/ChIP/bin/macs2" 

# P-VAL
p_val = 0.01

# SCRIPT INFO
script_name = f"Create_PseudoReps_Align_and_Call_Peaks.sh"
bash_header = ["#!/bin/bash", "#SBATCH -p memory", "#SBATCH --job-name pseudo", 
               "#SBATCH -o %j.out", "#SBATCH -e %j.err", "## ACTIVATE CONDA", 
               'eval "$(conda shell.bash hook)"', 'conda activate ChIP',
              ]

# REFERENCES
ref_dir = "/groups/doudna/team_resources/shared_databases/human_reference_genomes/hg38/GENCODE_GRCh38p14_v44/GRCh38_PRI_bt"

## CREATE PSEUDOREPS
pseudorep_bams = {}
target_depth = 50000000.0

# For each original BAM...
for bam in bams:
    base_id = bam.split("/")[-1].replace(bam_suffix, "")
    print(base_id)
    temp_bams = []
    bash_header.append(f"## {base_id}:")

    
    # CLEAN ORIGINAL BAM: Remove unaligned reads and format BAM
    # Extract aligned reads only and store new bam
    aligned_bam = f'{pseudo_bam_dir}{base_id}_aligned.bam'
    filter_aligned_cmd = f'samtools view -@ {threads} -b -F 4 {bam} | samtools sort -@ {threads} -o {aligned_bam}'
    bash_header.append(filter_aligned_cmd)

    # Index clean alignment
    index_cmd = f'samtools index -@ {threads} {aligned_bam}'
    bash_header.append(index_cmd)

    # Write mapped reads to Fastq
    r1 = f'{pseudo_bam_dir}{base_id}_R1.fq.gz'
    r2 = f'{pseudo_bam_dir}{base_id}_R2.fq.gz'
    fastq_cmd = f'samtools fastq -@ {threads} -1 {r1} -2 {r2} {aligned_bam}'
    bash_header.append(fastq_cmd)

    # Repair read order in mapped-read fastq
    r1_repaired = f'{pseudo_bam_dir}{base_id}_R1_repaired.fq.gz'
    r2_repaired = f'{pseudo_bam_dir}{base_id}_R2_repaired.fq.gz'
    repair_cmd = f'{repair_dir} in1={r1} in2={r2} out1={r1_repaired} out2={r2_repaired} repair'
    bash_header.append(repair_cmd)

    
    # For each pseudo-rep...
    for j in range(0, n_reps):
        bash_header.append(f"# {base_id} Pesudo {j}:")
        seed = seeds[j]
        
        # Get subsample fraction
        fraction = target_depth/mapped_dict[base_id]

        # Create pseudo-rep fastqs
        r1_sub = f'{pseudo_bam_dir}{base_id}_pseudo{j}_R1.fq.gz'
        r2_sub = f'{pseudo_bam_dir}{base_id}_pseudo{j}_R2.fq.gz'
        
        r1_sub_cmd = f'seqtk sample -s{seed} {r1_repaired} {fraction} | gzip > {r1_sub}'
        bash_header.append(r1_sub_cmd)
        
        r2_sub_cmd = f'seqtk sample -s{seed} {r2_repaired} {fraction} | gzip > {r2_sub}'
        bash_header.append(r2_sub_cmd)
        
        
        # Create pseudo-rep BAM:
        pseudoreplicate_bam = f"{pseudo_bam_dir}{base_id}_pseudo{j}{bam_suffix}"
        bowtie_align = (f"bowtie2 -x {ref_dir} -1 {r1_sub} -2 {r2_sub} -p {threads} "
                        f"--no-discordant --local --no-mixed --maxins 1000 | " # Minimize large gaps (confuses MACS2)
                        f"samtools view -bSh - | samtools sort -@ {threads} -o {pseudoreplicate_bam}")
        bash_header.append(bowtie_align)
        
        # Index pseudo-rep Bam
        index_cmd = f'samtools index -@ {threads} {pseudoreplicate_bam}'
        bash_header.append(index_cmd)

        # Store bam
        temp_bams.append(pseudoreplicate_bam)

    # Clean up
    clean_cmd = f'rm {r1} {r2} {aligned_bam}*'
    bash_header.append(clean_cmd)
    
    # Store bams
    pseudorep_bams[base_id] = temp_bams


#### PEAK CALLING ####

# BACKGROUND SAMPLES FOR MACS
background_dict = {'ChIP__KMW_060324_dWT_UCLA':pseudorep_bams['ChIP__KMW_060324_dWTapo_UCLA'],
                   'ChIP__KMW_060324_dSpRY_UCLA':pseudorep_bams['ChIP__KMW_060324_dSpRYapo_UCLA']
                  }

## CALL PEAKS AND DEDUP BAMS
picard_dir = "/shared/software/gatk/v4.1.5/bin/gatk"
all_ipd_bams = [bam for l in pseudorep_bams.values() for bam in l if "apo" not in bam]

bash_header.append(f"#### CALL PEAKS ####")

for bam in all_ipd_bams:
    # Collect Vars
    if "apo" in bam: # Skip Apo: double-check
        continue
    base = bam.split("/")[-1].replace("_sorted.bam", "").split("_pseudo")[0] # For background sample lookup
    sample_id = bam.split("/")[-1].replace("_sorted.bam", "") # For output naming
    input_ctrl_bam = background_dict[base]

    # CALL PEAKS
    bash_header.append(f"## MACS2 {sample_id}")
    print(f"## {sample_id}")
    
    macs2_peaks = (f"{macs2_dir} callpeak -t {bam} -c {' '.join(input_ctrl_bam)} "
                   f"-g hs -n {sample_id} -f BAM -B --SPMR -p {p_val} --outdir {peak_dir}  "
                   f"2> {peak_dir}{sample_id}_macs2.log")
    bash_header.append(macs2_peaks)

    # DEDUP
    out_bam = bam.replace(".bam", "_nodups.bam").replace(bam_dir, dedup_dir)
    metric_file = f"{dedup_dir}{sample_id}_nodups_report.txt"
    rm_dup_cmd = (f"{picard_dir} MarkDuplicates -I {bam} -O {out_bam} -M {metric_file} "
                  f"--REMOVE_DUPLICATES true")
    print(rm_dup_cmd)
    bash_header.append(rm_dup_cmd)


# WRITE SCRIPT
with open(script_name, "w") as f:
    f.write("\n".join(bash_header))
    
#os.system(f'sbatch {script_name}') #JID: 499694

In [4]:
#### MAKE BIGWIGS ####
## SETUP

# BASH
script_name = "Pseudo_BigWigs.sh"

bash_header = [
    "#!/bin/bash",
    "#SBATCH -p standard",
    "#SBATCH --job-name pseudo_bw",
    "#SBATCH -o %j.out",
    "#SBATCH -e %j.err",
    "## ACTIVATE CONDA",
    'eval "$(conda shell.bash hook)"',
    'conda activate ChIP',
]

## SETUP COMMANDS
#for bam in all_ipd_bams:
for bam in [pseudo_bam_dir+x for x in os.listdir(pseudo_bam_dir) if x.endswith(bam_suffix)]:
    sample_id = bam.split("/")[-1].replace("_sorted.bam", "") # For output naming
    bash_header.append(f"## {sample_id}")
    
    # RAW DEDUP
    dedup_raw_bw = f"{pseudo_bam_dir}{sample_id}_RAW_DEDUP.bigWig"
    bw_raw_dedup = (f"bamCoverage -b {bam} -o {dedup_raw_bw} "
                    f"--numberOfProcessors {threads} --ignoreDuplicates --binSize 1")
    if not os.path.exists(dedup_raw_bw):
        bash_header.append(bw_raw_dedup)    
    else:
        bash_header.append("#"+bw_raw_dedup)

    # CPM DEDUP
    dedup_bw = f"{pseudo_bam_dir}{sample_id}_CPM_Norm_DEDUP.bigWig"
    bw_dedup = (f"bamCoverage -b {bam} -o {dedup_bw} --normalizeUsing CPM "
                f"--numberOfProcessors {threads} --ignoreDuplicates --binSize 1")
    if not os.path.exists(dedup_bw):
        bash_header.append(bw_dedup)
    else:
        bash_header.append("#"+bw_dedup)

    # RAW
    raw_bw = f"{pseudo_bam_dir}{sample_id}_RAW.bigWig"
    bw_cmd = (f"bamCoverage -b {bam} -o {raw_bw} "
                f"--numberOfProcessors {threads}  --binSize 1")
    if not os.path.exists(raw_bw):
        bash_header.append(bw_cmd)
    else:
        bash_header.append("#"+bw_cmd)
        
        
# WRITE SCRIPT
with open(script_name, "w") as f:
    f.write("\n\n".join(bash_header))

#os.system(f'sbatch {script_name}') # JID: 499761, 500201

In [5]:
#### SNR ####
'''
Signal-to-Noise = num_reads_in_peaks / num_reads_outside_peaks
'''

narrowpeak_cols = ["Chrom", "Start", "End", "Peak_Name", "Score", "Strand", "SignalValue", "pValue", "qValue", "Summit_Loc"]
spry_apo_dedups = [f'{dedup_dir}ChIP__KMW_060324_dSpRYapo_UCLA_pseudo{j}{dedup_suffix}' for j in range(0,3)]
wt_apo_dedups = [f'{dedup_dir}ChIP__KMW_060324_dWTapo_UCLA_pseudo{j}{dedup_suffix}' for j in range(0,3)]

bam_to_peak_dict = {f"{dedup_dir}{sample_id}_pseudo{j}{dedup_suffix}":f"{peak_dir}{sample_id}_pseudo{j}{peak_suffix}" \
                    for sample_id in sample_ids+['EMX1_gRNA3'] for j in range(0,3)}

background_pseudo_dedup_dict = {'ChIP__KMW_060324_dSpRY_UCLA':spry_apo_dedups,
                                'ChIP__KMW_060324_dWT_UCLA': wt_apo_dedups,
                               }
snrs = {}

for bam,peak in bam_to_peak_dict.items():
    if not os.path.exists(bam+".bai"):
        os.system(f'samtools index -@ 12 {bam}')
        
    # Sample id
    sample_id = bam.split("/")[-1].replace(dedup_suffix, "")
    print(f'## PROCESSING: {sample_id}')
    
    # Get Apo files
    apo_bam_file = background_pseudo_dedup_dict[sample_id.split("_pseudo")[0]]
    print("##", sample_id, apo_bam_file)

    # READ SAMPLE PEAKS
    sample_peaks_df = pd.read_csv(peak, sep='\t', header=None, names=narrowpeak_cols)
    
    
    # Calculate Signal-to-Noise ratio
    signal_to_noise = calculate_signal_to_noise(bam, sample_peaks_df)
    print(f"Signal-to-Noise Ratio: {signal_to_noise}")
    snrs[sample_id] = signal_to_noise

# Get Rep SNR means:
wt_snr_rep_mean = np.mean([v for k,v in snrs.items() if "_dWT_" in k])*100
spry_rep_mean = np.mean([v for k,v in snrs.items() if "_dSpRY_" in k])*100

In [6]:
#### IDR ####
peak_rep_dict = {x:[f"{peak_dir}{x}_pseudo{j}{peak_suffix}" for j in range(0,3)] for x in background_dict.keys()}
idr_dir = f"{peak_dir}IDR/"
idr_threshold=0.05

## CALCULATE IDR
# BASH
script_name = "Pseudo_IDR.sh"

bash_header = [
    "#!/bin/bash",
    "#SBATCH -p standard",
    "#SBATCH --job-name pseudo_idr",
    "#SBATCH -o %j.out",
    "#SBATCH -e %j.err",
    "## ACTIVATE CONDA",
    'eval "$(conda shell.bash hook)"',
    'conda activate idr',
]
idr_results_files = []
for treatment,rep_list in peak_rep_dict.items():
    pairwise_combinations = itertools.combinations(rep_list, 2)

    for pair in pairwise_combinations:
        idr_results_file = f"{idr_dir}{treatment}___{'_'.join([x.split('/')[-1].split(peak_suffix)[0] for x in pair])}"
        idr_results_files.append(idr_results_file)
        
        idr_cmd = (f"idr --samples {' '.join(pair)} "
                   f"--output-file {idr_results_file} "
                   f"--plot "
                   f"--idr-threshold {idr_threshold}" 
                  )
        
        if not os.path.exists(idr_results_file):
            bash_header.append(idr_cmd)
        else:
            bash_header.append("#"+idr_cmd)

# WRITE SCRIPT
with open(script_name, "w") as f:
    f.write("\n".join(bash_header))

#os.system(f'sbatch {script_name}') # JID: 500668

## COLLECT REPRODUCIBLE PEAK COUNTS
idr_column_names = ["chrom",            # Chromosome name
                    "chromStart",       # Start position of the peak on the chromosome
                    "chromEnd",         # End position of the peak on the chromosome
                    "name",             # Name assigned to the peak region
                    "score",            # Scaled IDR score
                    "strand",           # Strand information
                    "signalValue",      # Measurement of enrichment for the region
                    "p-value",          # Merged peak p-value
                    "q-value",          # Merged peak q-value
                    "summit",           # Position of the peak summit
                    "localIDR",         # Local IDR score
                    "globalIDR",        # Global IDR score
                    "rep1_chromStart",  # Start position of the peak in replicate 1
                    "rep1_chromEnd",    # End position of the peak in replicate 1
                    "rep1_signalValue", # Signal value for replicate 1
                    "rep1_rank",        # Rank of the peak in replicate 1
                    "rep2_chromStart",  # Start position of the peak in replicate 2
                    "rep2_chromEnd",    # End position of the peak in replicate 2
                    "rep2_signalValue", # Signal value for replicate 2
                    "rep2_rank"         # Rank of the peak in replicate 2
                   ]

reproducible_peak_counts_dict = {}
treatments = []
for idr_results in idr_results_files:
    combo = idr_results.split("/")[-1].replace("ChIP__KMW_060324_","").split("___")[-1]
    treatments.append(idr_results.split("/")[-1].split("___")[0])
    idr_results_df = pd.read_csv(idr_results, sep = "\t", header = None, names = idr_column_names)
    reproducible_peak_counts_dict[combo] = idr_results_df.shape[0]

reproducible_peak_counts_df = pd.DataFrame({"Pair":reproducible_peak_counts_dict.keys(), 
                                            "N_Reproduced":reproducible_peak_counts_dict.values(), 
                                            "Treatment":treatments})
agg_df = reproducible_peak_counts_df.groupby("Treatment").agg(Mean_Reproducible=("N_Reproduced","mean"), 
                                                              Std=("N_Reproduced","std")).reset_index()
# Peak Counts (from IDR results files)
peak_counts = {'dWT_UCLA_pseudo1': 44240,
               'dWT_UCLA_pseudo0': 44633,
               'dSpRY_UCLA_pseudo2': 45584,
               'dSpRY_UCLA_pseudo1': 44744,
               'dSpRY_UCLA_pseudo0': 45160,
               'dWT_UCLA_pseudo2': 44297}
peak_summary_df = reproducible_peak_counts_df.loc[(reproducible_peak_counts_df.Pair.str.contains("_dWT_"))|\
                                          (reproducible_peak_counts_df.Pair.str.contains("_dSpRY_"))]
peak_summary_df["Rep1"] = peak_summary_df["Pair"].apply(lambda x: x.split("_dWT")[0].split("_dSpRY")[0])
peak_summary_df["Rep2"] = ["dWT", "dWT", "dWT", "dSpRY", "dSpRY", "dSpRY"] + peak_summary_df["Pair"].apply(lambda x: x.split("_dWT")[1] if "_dWT" in x else x.split("_dSpRY")[1])
peak_summary_df["Rep2"] = peak_summary_df["Rep2"]
peak_summary_df["Rep_Rate"] = 100*peak_summary_df["N_Reproduced"]/[np.mean(y) for y in zip([peak_counts[x] for x in peak_summary_df["Rep1"]],[peak_counts[x] for x in peak_summary_df["Rep2"]])]

peak_summary_df.groupby("Treatment").agg(Avg_Rep_Rate=("Rep_Rate", "mean"))


In [7]:
#### RPKM: +/-25bp Guide with Seed-PAM Matches ####



## REGEX SEED-PAM REGIONS
# Seed&PAM Regular expressions:
regex_dict = {"SpRY": ["[ATGC]{15}AAGAA[ATGC][AG][ATGC]",
                       f'[ATGC][TC][ATGC]{reverse_complement("AAGAA")}[ATGC]{{15}}' # Reverse complement 
                      ], 
              "WT":["[ATGC]{15}AAGAA[ATGC]GG",
                     f'CC[ATGC]{reverse_complement("AAGAA")}[ATGC]{{15}}' # Reverse complement 
                   ]
             }

# Generate bed files of Seed&PAM matches
match_dir = f"{out_dir}Pattern_Matches/"
for cas,regex_pair in regex_dict.items():
    find_hits_and_create_bed(cas,   
                             regex_pair, 
                             bed_path = f"{match_dir}{cas}_hits_SEEDPAM.bed"
                            )

# Clean/sort beds for processing
beds = [f"{match_dir}WT_hits_SEEDPAM.bed", f"{match_dir}SpRY_hits_SEEDPAM.bed"]
new_bed_suffix = "_sorted_v25bpWindow.bed"
clean_beds = []
for bed in beds:
    clean_bed = f'{bed.replace(".bed", new_bed_suffix)}'
    clean_beds.append(clean_bed)
    sort_bed_file(bed, clean_bed, window=25) # Sort bed and adjust for +/-25bp around Seed&PAM

# Create bed file for intervals/regions WITHOUT Seed&PAM pattern matches
faidx_path = "/groups/doudna/team_resources/shared_databases/human_reference_genomes/hg38/GENCODE_GRCh38p14_v44/GRCh38_PRI.fa.fai"
no_match_suffix = "_noSeedPAMMatch.bed"
no_seedpam_match_beds = []
for bed in clean_beds:
    no_match_bed = f'{bed.replace(".bed", no_match_suffix)}'
    no_seedpam_match_beds.append(no_match_bed)
    comp_cmd = f"bedtools complement -i {bed} -g {faidx_path} > {no_match_bed}"
    os.system(comp_cmd)



## GET COVERAGE FOR REGIONS OF INTEREST  
def calculate_coverage(bam_file, bed_file, output_file="ROI_Coverage.bed"):
    '''
    Get read coverage from bam for intervals within a bed file
    '''
    if not output_file:
        output_file = f"{os.path.splitext(bam_file)[0]}_coverage.bed"
    cmd = f"bedtools coverage -a {bed_file} -b {bam_file} -counts > {output_file}"
    subprocess.run(cmd, shell=True)

pseudorep_dedup_bam_list = [f'{dedup_dir}ChIP__KMW_060324_dWT_UCLA_pseudo2_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dSpRY_UCLA_pseudo2_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dWT_UCLA_pseudo1_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dSpRY_UCLA_pseudo1_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dWT_UCLA_pseudo0_sorted_nodups.bam',
                            f'{dedup_dir}hIP__KMW_060324_dSpRY_UCLA_pseudo0_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dWTapo_UCLA_pseudo0_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dSpRYapo_UCLA_pseudo0_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dWTapo_UCLA_pseudo2_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dSpRYapo_UCLA_pseudo1_sorted_nodups.bam',
                            f'{dedup_dir}DEDUP/ChIP__KMW_060324_dWTapo_UCLA_pseudo1_sorted_nodups.bam',
                            f'{dedup_dir}ChIP__KMW_060324_dSpRYapo_UCLA_pseudo2_sorted_nodups.bam'
                 ]
hit_coverage = []
nonhit_coverage = []
hit_bed_cov_suffix = f'_coverage_hit_SEEDPAM.bed'
nonhit_bed_cov_suffix = f'_NoMatchIntervals_SEEDPAM.bed'

for bam in pseudorep_dedup_bam_list:

    if "SpRY" in bam:
        cas_beds = [x for x in no_seedpam_match_beds+clean_beds if "SpRY" in x]
    else:
        cas_beds = [x for x in no_seedpam_match_beds+clean_beds if "WT" in x]

    for bed in cas_beds:
        cas_id = bed.split("/")[-1].split("_")[0]
        if bed.endswith(no_match_suffix):
            out_cov = bam.replace(".bam", f'___{cas_id}PAM__'+nonhit_bed_cov_suffix)
            nonhit_coverage.append(out_cov)
        else:
            out_cov = bam.replace(".bam", hit_bed_cov_suffix)
            hit_coverage.append(out_cov)

        calculate_coverage(bam, bed, output_file=out_cov)



## Exclude BLACKLIST REGIONS
'''
Exclude blacklisted regions of hg38 from RPKM quantification
'''

blacklist = f"{match_dir}hg38-blacklist.v2.bed" # Blacklist file accroding to Methods

for bed in glob.glob(f"{dedup_dir}*coverage_hit_SEEDPAM.bed")+\
           glob.glob(f"{dedup_dir}*NoMatchIntervals_SEEDPAM.bed"):
   
    out_bed = bed.replace(".bed", "_filtered.bed")
    bedtools_cmd = f"bedtools intersect -a {bed} -b {blacklist} -v > {out_bed}"
    if not os.path.exists(out_bed):
        print(bedtools_cmd)



## RUN RPKM CALCULATION: Use apo background subtraction
# Collect Number mapped reads
mapped_bam_reads = {}
for bam in pseudorep_dedup_bam_list:
    sample_id = bam.split("/")[-1].split("_sorted_nodups.bam")[0]
    bamfile = pysam.AlignmentFile(bam, "rb")
    total_reads = bamfile.mapped
    bamfile.close()
    mapped_bam_reads[sample_id] = total_reads
    print(sample_id, total_reads)
    
experimental_setup = {'SpRY_SpRYPAM':['SpRY_SpRYPAM', 'SpRYapo_SpRYPAM'],  # Get RPKM for Seed&SpRyPAM regions subtract out Apo background
                      'SpRY_WTPAM':['SpRY_WTPAM', 'SpRYapo_WTPAM'], # Get RPKM for Seed&WTPAM regions, ract out Apo background
                      'WT_WTPAM': ['WT_WTPAM', 'WTapo_WTPAM'], # Get RPKM for Seed&WTPAM regions, subtract out Apo background
                      'WT_SpRYPAM': ['WT_SpRYPAM', 'WTapo_SpRYPAM'] # Get RPKM for Seed&SpRYPAM regions, subtract out Apo background
                     }
# Get RPKM for Seed&PAM pattern
SEEDPAM_hit_results = process_chip_data(f"{dedup_dir}*coverage_hit_SEEDPAM_filtered.bed", 
                                        mapped_bam_reads, experimental_setup)
# Get RPKM for reverse-complement Seed&PAM pattern
SEEDPAM_comp_results = process_chip_data(f"{dedup_dir}*NoMatchIntervals_SEEDPAM_filtered.bed",
                                         mapped_bam_reads, experimental_setup)


## PLOT
hit_comp_pairs = {"SEEDPAM":[SEEDPAM_hit_results, SEEDPAM_comp_results]}
xs = ["SpRY_SpRYSeedPAM", "SpRY_NoSpRYSeedPAM", "SpRY_WTSeedPAM", "SpRY_NoWTSeedPAM",
      "WT_WTSeedPAM", "WT_NoWTSeedPAM", "WT_SpRYSeedPAM", "WT_NoSpRYSeedPAM"
     ] # Format X-axis labels

treatment_filter = ['SpRY_SpRYPAM','SpRY_SpRYPAM_NONE','WT_WTPAM','WT_WTPAM_NONE']

for analysis_id,hit_comp in hit_comp_pairs.items():
    comp = hit_comp[1]
    all_data = pd.concat([hit_comp[0], comp])
    all_data.sort_values("Treatment", inplace = True)

    fig, axs = plt.subplots(figsize=(3, 4))
    plot_bar(axs, 
             all_data.Treatment.to_list(),
             all_data["Avg_RPKM"].to_list(), 
             "", 
             "RPKM",
             get_color, 
             yerr=[0 if np.isnan(x) else x for x in all_data["Std_RPKM"].to_list()]
            )
    
    # Format Legend
    legend_elements = [
        plt.Line2D([0], [0], color='mediumorchid', lw=4, label='SpRY'),
        plt.Line2D([0], [0], color='grey', lw=4, label='WT')
    ]
    plt.tight_layout(rect=[0, 0, 1, 0.85])  # Adjust the margin
    fig.legend(handles=legend_elements, loc='upper center', bbox_to_anchor=(0.5, 0.94), ncol=3)
    fig.suptitle(f"{analysis_id.replace('173','100').replace('73','50')}", fontsize=16, y=1.0)
    
    # Save
    plt.savefig(f"{analysis_id}bp_Clean.svg")